In [1]:
/*
Question 1:
Summarize this dataset. The actual analytics and metrics are left up to your discretion.
This problem is left intentionally unstructured, so just include a file or notebook that
describes quantitatively what this dataset contains.
*/

import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
.appName("StreetTreeCensus")
.getOrCreate()

val treeDF = spark.read.format("csv").option("header","true").load("/host/Users/projects/juypterNotebook/DisneyStreaming/2015StreetTreesCensus_TREES.csv")
treeDF.coalesce(4)
treeDF.cache
treeDF.createOrReplaceTempView("treeDFTable")


val totalTreeCount = treeDF.count
println(s"Total Number Of Trees: ${totalTreeCount}")

println


val meanOfDiameter = treeDF.select(mean("tree_dbh"), mean("stump_diam")).collectAsList.get(0)
println(s"Mean of Breast Height of Tree, Stump Diameter:  ${meanOfDiameter.get(0)}, ${meanOfDiameter.get(1)}")

val stdOfDiameter = treeDF.select(stddev("tree_dbh"), stddev("stump_diam")).collectAsList.get(0)
println(s"Standard Deviation of Breast Height of Tree, Stump Diameter: ${stdOfDiameter.get(0)}, ${stdOfDiameter.get(1)}")



println

val totalTreeOnCurb = treeDF.select("curb_loc").filter($"curb_loc" === "OnCurb").count
val totalTreeOffsetFromCurb = treeDF.select("curb_loc").filter($"curb_loc" === "OffsetFromCurb").count
println(s"Total Tree On Curb, Total Tree Offset From Curb:  ${totalTreeOnCurb}, ${totalTreeOffsetFromCurb}")

println

val totalDeadTreeCount = treeDF.select("status").where($"status" === "Dead").count
val totalAliveTreeCount = treeDF.select("status").where($"status" === "Alive").count
val totalStumpTreeCount = treeDF.select("status").where($"status" === "Stump").count

println(s"Dead Tree, Alive Tree, Stump Tree:  ${totalDeadTreeCount}, ${totalAliveTreeCount}, ${totalStumpTreeCount}")


println


val goodTreeCount = treeDF.select("health").where($"health" === "Good").count
val fairTreeCount = treeDF.select("health").where($"health" === "Fair").count
val poorTreeCount = treeDF.select("health").where($"health" === "Poor").count

println(s"Good Tree, Fair Tree, Poor Tree:  ${goodTreeCount}, ${fairTreeCount}, ${poorTreeCount}")

println("Top 10 borough: Trees with 'Good' health")
treeDF.where($"health" === "Good").groupBy("boroname").agg(count("health").alias("count")).orderBy(desc("count")).show(10)

println("Top 10 borough: Trees with 'Poor' health")
treeDF.where($"health" === "Poor").groupBy("boroname").agg(count("health").alias("count")).orderBy(desc("count")).show(10)


println(f"Percentage of Trees Having Good Health :  ${(goodTreeCount.toDouble/totalTreeCount) * 100}%.2f%%")
println(f"Percentage of Trees Having Poor Health :  ${(poorTreeCount.toDouble/totalTreeCount) * 100}%.2f%%")

println("Top 20 Tree Name by Thier Count")
treeDF.groupBy($"spc_common".alias("Tree Name")).agg(count($"spc_common").alias("Count")).sort(desc("Count")).show

val helpfulGuardsCount = treeDF.select("guards").where($"guards" === "Helpful").count
val harmfulGuardsCount = treeDF.select("guards").where($"guards" === "Harmful").count
val neitherHelpfulNorHarmfulCount = treeDF.select("guards").where($"guards" === "None").count
val unsureCount = treeDF.select("guards").where($"guards" === "Unsure").count
val totalGaurdsCount = treeDF.select("guards").where($"guards".isNotNull).count

println(s"Total Gaurds , Helpful Gaurds , Harmful Gaurds , Niether Helpful nor Harmful, Unsure: $totalGaurdsCount, $helpfulGuardsCount, $harmfulGuardsCount, $neitherHelpfulNorHarmfulCount, $unsureCount")

println(f"Percentage of Helpful Gaurd  :  ${(helpfulGuardsCount.toDouble/totalGaurdsCount) * 100}%.2f%%")
println(f"Percentage of Harmful Gaurd  :  ${(harmfulGuardsCount.toDouble/totalGaurdsCount) * 100}%.2f%%")
println(f"Percentage of Neither Helpful not Harmful Gaurd  :  ${(neitherHelpfulNorHarmfulCount.toDouble/totalGaurdsCount) * 100}%.2f%%")


val treeWithRootStoneProblem =  treeDF.select("root_stone").where($"root_stone" === "Yes").count
val treeWithRootGrateProblem =  treeDF.select("root_grate").where($"root_grate" === "Yes").count
val treeWithRootOtherProblem =  treeDF.select("root_other").where($"root_other" === "Yes").count

val treeWithRootProblem:Double = treeWithRootStoneProblem + treeWithRootGrateProblem + treeWithRootOtherProblem
println(f"Percentable of Tree With Root Problem:  ${(treeWithRootStoneProblem.toDouble/totalTreeCount) * 100}%.2f")


val treeWithTrunkWireProblem =  treeDF.select("trnk_wire").where($"trnk_wire" === "Yes").count
val treeWithTrunkLightProblem =  treeDF.select("trnk_light").where($"trnk_light" === "Yes").count
val treeWithTrunkOtherProblem =  treeDF.select("trnk_other").where($"trnk_other" === "Yes").count

val treeWithTrunkProblem:Double = treeWithTrunkWireProblem + treeWithTrunkLightProblem + treeWithTrunkOtherProblem
println(f"Percentable of Tree With Trunk Problem:  ${(treeWithTrunkProblem.toDouble/totalTreeCount) * 100}%.2f")


val treeWithBranchLighProblem =  treeDF.select("brnch_ligh").where($"brnch_ligh" === "Yes").count
val treeWithBranchShowProblem =  treeDF.select("brnch_shoe").where($"brnch_shoe" === "Yes").count
val treeWithBranchOtherProblem =  treeDF.select("brnch_othe").where($"brnch_othe" === "Yes").count

val treeWithBranchProblem:Double = treeWithBranchLighProblem + treeWithBranchShowProblem + treeWithBranchOtherProblem
println(f"Percentable of Tree With Branch Problem:  ${(treeWithBranchProblem.toDouble/totalTreeCount) * 100}%.2f")


println("Top 20 Tree Count by zip_city")
treeDF.groupBy($"zip_city".alias("Zip City")).agg(count($"tree_id").alias("Count")).sort(desc("Count")).show


println("Bottom 20 Tree Count by zip_city")
treeDF.groupBy($"zip_city".alias("Zip City")).agg(count($"tree_id").alias("Count")).sort(asc("Count")).show

println("Tree Count by borough ")
treeDF.groupBy($"boroname".alias("borough")).agg(count($"tree_id").alias("Count")).sort(desc("Count")).show



Total Number Of Trees: 683788                                                   

Mean of Breast Height of Tree, Stump Diameter:  11.27978701000895, 0.43246298560372515
Standard Deviation of Breast Height of Tree, Stump Diameter: 8.723042268549444, 3.2902407401961704

Total Tree On Curb, Total Tree Offset From Curb:  656896, 26892

Dead Tree, Alive Tree, Stump Tree:  13961, 652173, 17654

Good Tree, Fair Tree, Poor Tree:  528850, 96504, 26818
Top 10 borough: Trees with 'Good' health
[Stage 24:====================================================> (195 + 4) / 200]+-------------+------+
|     boroname| count|
+-------------+------+
|       Queens|194008|
|     Brooklyn|138212|
|Staten Island| 82669|
|        Bronx| 66603|
|    Manhattan| 47358|
+-------------+------+

Top 10 borough: Trees with 'Poor' health
+-------------+-----+                                                           
|     boroname|count|
+-------------+-----+
|       Queens| 9417|
|     Brooklyn| 6459|
|Staten Island

spark = org.apache.spark.sql.SparkSession@1ac241a3
treeDF = [created_at: string, tree_id: string ... 40 more fields]
totalTreeCount = 683788
meanOfDiameter = [11.27978701000895,0.43246298560372515]
stdOfDiameter = [8.723042268549444,3.2902407401961704]
totalTreeOnCurb = 656896
totalTreeOffsetFromCurb = 26892
totalDeadTreeCount = 13961
totalAliveTreeCount = 652173
totalStumpTreeCount = 17654
goodTreeCount = 528850
fairTreeCount = 96504
poorTreeCount = 26818
helpfulGuardsCount = 51866
harmfulGuardsCount = 20252
neitherHelpfulNorHarmfulCount = 572306


unsureCount:...


572306